# Subprocess replacements generated by Bing Open AI

https://www.bing.com/search?pglt=163&q=ipython+show+line+by+line+output+from+external+command&cvid=6ff48e255bf744a6a7d6b61a73ff6f15&aqs=edge..69i57j69i64j69i11004.13666j0j1&FORM=ANNAB1&PC=U531&ntref=

**timout** - has a bug - it requires an output for a timoout to occur :-( (it does not work as expected)

many discussions on SO - but no cross platform solution
https://stackoverflow.com/questions/3140189/subprocess-popen-stdout-reading-stdout-in-real-time-again?rq=1


In [1]:
# import subprocess
# from IPython.core.interactiveshell import InteractiveShell

# InteractiveShell.ast_node_interactivity = "all"


# def show_output(cmd):
#     process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#     while True:
#         output = process.stdout.readline()
#         if output == b"" and process.poll() is not None:
#             break
#         if output:
#             print(output.decode("utf-8").strip())
#     rc = process.poll()
#     return rc

In [83]:
# updated version of the code that can handle a timeout of 60 seconds if no output is received from the process:

import subprocess
import time
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


def show_output(cmd, timeout=60):
    print(f"per line, with timeout of {timeout} seconds")
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    start_time = time.time()
    while True:
        output = process.stdout.readline()
        if output == b"" and process.poll() is not None:
            break
        if output:
            print(output.decode("utf-8").strip())
            start_time = time.time()
        elif time.time() - start_time > timeout:
            print(f"No output received for {timeout} seconds. Terminating process.")
            process.terminate()
            break
    rc = process.poll()
    return rc

In [17]:
# per character
# timout - has a bug - it requires an output for a timoout to occur :-( (it does not work as expected)

import subprocess
import time
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


def show_output(cmd, timeout=60):
    print(f"per char , with timeout of {timeout} seconds")
    assert timeout > 0
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    start_time = time.time()
    assert process.stdout is not None
    while True:
        output = process.stdout.read(1)
        if output == b"" and process.poll() is not None:
            break
        if output:
            print(output.decode("utf-8"), end="", flush=True)
            start_time = time.time()
        elif time.time() - start_time > timeout:
            print(f"\nNo output received for {timeout} seconds. Terminating process.")
            process.terminate()
            break
    return process.poll()

In [78]:
# per character
# timout - fixed

from threading import Timer
import subprocess
import time
from typing import List, Union
from IPython.core.interactiveshell import InteractiveShell

from loguru import logger as log


def show_output(cmd: Union[List[str], str], timeout: Union[int, float] = 120, shell: bool = False):
    InteractiveShell.ast_node_interactivity = "all"
    try:
        log.info(f"per char , with timeout of {timeout} seconds")
        assert timeout > 0
        try:
            process = subprocess.Popen(
                cmd, shell=shell, stdout=subprocess.PIPE, stderr=subprocess.PIPE
            )  # ,  universal_newlines=True)
        except FileNotFoundError as e:
            raise FileNotFoundError(f"Failed to start {cmd[0]}") from e

        assert process.stdout is not None

        def timed_out():
            process.kill()
            log.warning(f"Command {cmd} timed out after {timeout} seconds")
            # print(f"Command {cmd} timed out after {timeout} seconds")

        process_timer = Timer(timeout, timed_out)
        process_timer.start()

        while True:
            output = process.stdout.read(1)
            if output == b"" and process.poll() is not None:
                break
            if output:
                print(output.decode("utf-8"), end="", flush=True)

        return process.poll()
    finally:
        # set handling back to the default value
        InteractiveShell.ast_node_interactivity = "last"

         11 function calls in 0.001 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1209(_handle_fromlist)
        3    0.000    0.000    0.000    0.000 typing.py:357(inner)
        1    0.000    0.000    0.000    0.000 typing.py:1354(__hash__)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.hash}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [84]:
%%prun -s tottime
show_output(["mpremote", "devs"], timeout=15)

per line, with timeout of 15 seconds
COM4 E661410403600E38 2e8a:0005 Microsoft None
 

         1077 function calls in 0.370 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      182    0.361    0.002    0.361    0.002 {method 'readline' of '_io.BufferedReader' objects}
        1    0.007    0.007    0.007    0.007 {built-in method _winapi.CreateProcess}
        2    0.001    0.000    0.001    0.000 socket.py:613(send)
        1    0.000    0.000    0.370    0.370 1426696547.py:10(show_output)
        1    0.000    0.000    0.370    0.370 <string>:1(<module>)
      181    0.000    0.000    0.000    0.000 {built-in method _winapi.WaitForSingleObject}
      183    0.000    0.000    0.000    0.000 subprocess.py:1516(_internal_poll)
        1    0.000    0.000    0.370    0.370 {built-in method builtins.exec}
      182    0.000    0.000    0.000    0.000 subprocess.py:1229(poll)
        1    0.000    0.000    0.000    0.000 subprocess.py:574(list2cmdline)
        2    0.000    0.000    0.000    0.000 {built-in me

In [70]:
show_output("ping google.com", timeout=1.2)


Pinging google.com [142.250.186.110] with 32 bytes of data:


Reply from 142.250.186.110: bytes=32 time=54ms TTL=116
Reply from 142.250.186.110: bytes=32 time=19ms TTL=116
WARNING  | __main__:timed_out:28 - Command ping google.com timed out after 1.2 seconds


1

In [71]:
show_output(["mpremote", "devs"], timeout=15)

COM4 E661410403600E38 2e8a:0005 Microsoft None


0

In [72]:
show_output("mpremote mip install github:josverl/micropython-stubber/mip/mpy_v6.json@board_stubber", timeout=30)

Install github:josverl/micropython-stubber/mip/mpy_v6.json
Installing github:josverl/micropython-stubber/mip/mpy_v6.json to /lib
Installing: /lib/createstubs.mpy
Installing: /lib/createstubs_mem.mpy
Installing: /lib/createstubs_db.mpy
Installing: /lib/modulelist.txt
Installing: /lib/board_info.csv
Done


0

WARNING  | __main__:timed_out:28 - Command ['mpremote', 'devs'] timed out after 15 seconds


In [19]:
# %%micropython

import time
import machine

led = machine.Pin(25, machine.Pin.OUT)

for i in range(5):
    print(i, end=", ")
    led.toggle()
    time.sleep(0.2)

print("\ndone")

['0, 1, 2, 3, 4, ', 'done']

In [22]:
show_output('mpremote exec "import __magic"', timeout=100)

per char , with timeout of 100 seconds
0, 1, 2, 3, 4, 
done


0

In [73]:
1
2
3

3

In [74]:
%macro __magic 1


Macro `__magic` created. To execute, type its name (without quotes).
=== Macro contents: ===
import select
import subprocess
import time
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


def show_output(cmd, timeout=60):
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    start_time = time.time()
    while True:
        # Check if the process has finished
        if process.poll() is not None:
            break
        # Check if there is data available to read
        output = process.stdout.read(1)
        if time.time() - start_time > timeout:
            print(f"\nNo output received for {timeout} seconds. Terminating process.")
            process.terminate()
            break
        elif output == b"" and process.poll() is not None:
            break
        if output:
            print(output.decode("utf-8"), end="", flush=True)
            start_time = time.time()
    rc 

WARNING  | __main__:timed_out:28 - Command mpremote mip install github:josverl/micropython-stubber/mip/mpy_v6.json@board_stubber timed out after 30 seconds


In [75]:
%hist -n 

   1:
import select
import subprocess
import time
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


def show_output(cmd, timeout=60):
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    start_time = time.time()
    while True:
        # Check if the process has finished
        if process.poll() is not None:
            break
        # Check if there is data available to read
        output = process.stdout.read(1)
        if time.time() - start_time > timeout:
            print(f"\nNo output received for {timeout} seconds. Terminating process.")
            process.terminate()
            break
        elif output == b"" and process.poll() is not None:
            break
        if output:
            print(output.decode("utf-8"), end="", flush=True)
            start_time = time.time()
    rc = process.poll()
    return rc
   2: show_output("ping google.com", timeout=1)
   3:
# 

In [57]:
%macro?

Docstring:
Define a macro for future re-execution. It accepts ranges of history,
filenames or string objects.

Usage:
  %macro [options] name n1-n2 n3-n4 ... n5 .. n6 ...

Options:

  -r: use 'raw' input.  By default, the 'processed' history is used,
  so that magics are loaded in their transformed version to valid
  Python.  If this option is given, the raw input as typed at the
  command line is used instead.
  
  -q: quiet macro definition.  By default, a tag line is printed 
  to indicate the macro has been created, and then the contents of 
  the macro are printed.  If this option is given, then no printout
  is produced once the macro is created.

This will define a global variable called `name` which is a string
made of joining the slices and lines you specify (n1,n2,... numbers
above) from your input history into a single string. This variable
acts like an automatic function which re-executes those lines as if
you had typed them. You just type 'name' at the prompt and the code
